<a href="https://colab.research.google.com/github/InTEGr8or/jupyter-fun/blob/master/nCov19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infection Rates per day

The percent rates in the sheet at the bottom are approximations. Data seems to be released about twice a day. Sometimes there is a lag, and they release it at different times.

Optimally, the time of the update would be taken into account and prorated per hour and multiplied by the number of hours difference but I am just starting to learn Python so there are probably a lot of improvements that could be made.


## This first section sets up the imports and some parsing functions.


In [0]:
try:
  import pygsheets
except:
  !pip install pygsheets
  import pygsheets
try:
  import numpy
except:
  !pip install numpy
  import numpy
from dateutil import parser
try:
  import requests
except:
  !pip install requests
  import requests

try:
  from bs4 import BeautifulSoup
except:
  !pip install beautifulsoup4
  from bs4 import BeautifulSoup
  
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
pd.set_option('display.max_rows', 100)

gc = gspread.authorize(GoogleCredentials.get_application_default())
source_url = "https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w"
def get_sheets(sheet_url):
  page = requests.get(sheet_url)
  worksheets = BeautifulSoup(page.text)
  # the requests just gets the ajax scripts at this point.
  # read from '//div[@id='sheets-viewport']/div //table'
  # thats all the sheets
  pass
# get_sheets(source_url)

def sdate(sheet_rows):
  header = sheet_rows[0]
  update_col = [i for i, item in enumerate(header) if "Update" in item or "update" in item]
  return parser.parse(sheet_rows[3][update_col[0]])

def rdate(sheet_rows):
  tdate = sdate(sheet_rows)
  return f'{tdate.strftime("%B")[:3]}{tdate.day}'

def rdtime(sheet_rows):
  tdate = parser.parse(sheet_rows[3][sheet_rows[0].index('Last Update')])


## Get The Sheets and Loop Through

In [0]:
#TODO: Open the public sheet in a way that doesn't require authentication, like the pd.read_csv() below.  
worksheets = gc.open_by_url(source_url).worksheets()
prows = worksheets[0].get_all_values()
prior_sheet = worksheets[0]
pdate = None
# df = pd.read_csv(source_url, dtype={0:'str', 1:'str', 2:'str'})
df = pd.DataFrame.from_records(prows[1:], columns=prows[0])
df.rename(columns={"Confirmed": rdate(prows)}, inplace=True)
for worksheet in worksheets:
  crows = worksheet.get_all_values()
  if sdate(crows).day != sdate(prows).day:
    try:
      prows = prior_sheet.get_all_values()
      prov_col = prows[0].index('Province/State')
      cnt_col = [i for i,item in enumerate(prows[0]) if "Country" in item][0]
      conf_col = prows[0].index('Confirmed')
      dt_col = rdate(crows)
      prior_dt_col = rdate(prows)
      df[f'{prior_dt_col}%'] = '0%'
      df[dt_col] = 0
      for i, df_row in df.iterrows():
        # TODO: Find the row for the df province
        for j, row in enumerate(prows):
          if row[cnt_col] == df_row['Country/Region'] and row[prov_col] == df_row['Province/State']:
            if row[conf_col].isnumeric():
              df.at[i, dt_col] = row[conf_col]
              df.at[i, f'{prior_dt_col}%'] = f'{(round(int(df_row[prior_dt_col]) / int(row[conf_col]) * 100) - 100)}%'
    except:
      print(prows[:2])

  prior_sheet = worksheet



## Print Out the Data


In [154]:
df.drop(labels='Last Update',axis=1)


,Province/State,Country/Region,Feb1,Deaths,Recovered,Feb1%,Jan31,Jan31%,Jan30,Jan30%,Jan29,Jan29%,Jan28,Jan28%,Jan27,Jan27%,Jan26,Jan26%,Jan25,Jan25%,Jan24,Jan24%,Jan23,Jan23%,Jan22,Jan22%,Jan21
0,Hubei,Mainland China,7153,249,168,0%,7153,23%,5806,18%,4586,29%,3554,31%,2714,91%,1423,34%,1052,44%,549,0%,549,24%,444,0%,0
1,Zhejiang,Mainland China,599,0,21,0%,537,0%,537,25%,428,45%,296,71%,173,35%,128,23%,104,68%,43,0%,43,59%,27,0%,0
2,Guangdong,Mainland China,535,0,14,0%,436,0%,393,11%,311,12%,241,16%,151,0%,146,32%,98,26%,53,0%,53,66%,32,0%,0
3,Henan,Mainland China,422,2,3,0%,352,0%,352,27%,278,35%,206,23%,168,31%,128,54%,83,159%,9,0%,9,80%,5,0%,0
4,Hunan,Mainland China,389,0,8,0%,332,0%,332,20%,277,25%,221,55%,100,0%,100,45%,69,60%,24,0%,24,167%,9,0%,0
5,Anhui,Mainland China,297,0,5,0%,237,0%,237,18%,200,32%,152,43%,106,51%,70,17%,60,54%,15,0%,15,67%,9,0%,0
6,Jiangxi,Mainland China,286,0,9,0%,240,0%,240,48%,162,49%,109,51%,72,50%,48,33%,36,100%,18,157%,7,0%,7,0%,0
7,Chongqing,Mainland China,247,1,3,0%,238,13%,206,13%,165,12%,147,11%,132,20%,110,47%,75,32%,27,0%,27,200%,9,0%,0
8,Sichuan,Mainland China,207,1,3,0%,177,0%,177,25%,142,31%,108,20%,90,30%,69,57%,44,57%,15,0%,15,88%,8,0%,0
9,Shandong,Mainland China,206,0,3,0%,184,0%,178,13%,145,12%,121,27%,87,16%,63,37%,39,86%,15,67%,9,50%,6,0%,0
